AES Encryption and Decryption

In [1]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding
import os

In [2]:
class AESCipher:
    def __init__(self, key):
        self.key = key
        self.backend = default_backend()

    def encrypt(self, plaintext):
        iv = os.urandom(16)
        cipher = Cipher(algorithms.AES(self.key), modes.CBC(iv), backend=self.backend)
        padder = padding.PKCS7(128).padder()
        padded_data = padder.update(plaintext) + padder.finalize()
        encryptor = cipher.encryptor()
        ciphertext = encryptor.update(padded_data) + encryptor.finalize()
        return iv + ciphertext

    def decrypt(self, ciphertext):
        iv = ciphertext[:16]
        cipher = Cipher(algorithms.AES(self.key), modes.CBC(iv), backend=self.backend)
        decryptor = cipher.decryptor()
        padded_plaintext = decryptor.update(ciphertext[16:]) + decryptor.finalize()
        unpadder = padding.PKCS7(128).unpadder()
        plaintext = unpadder.update(padded_plaintext) + unpadder.finalize()
        return plaintext

# Example usage
key = os.urandom(32)  # AES-256 key
aes_cipher = AESCipher(key)

plaintext = b"Hello, World!"
ciphertext = aes_cipher.encrypt(plaintext)
print("Encrypted:", ciphertext)

decrypted = aes_cipher.decrypt(ciphertext)
print("Decrypted:", decrypted)


Encrypted: b'\xe9\xef\xb2\x92\xda\xd1H\xe2;\xb2"8\xf4{\xe7\xa6\xd8\xfd\x03\x892b\x8b\x18\xe3ls\xd5Y\xd5\xc8\x80'
Decrypted: b'Hello, World!'


SHA256 for Data Hashing

In [3]:
import hashlib

def hash_data(data):
    sha256 = hashlib.sha256()
    sha256.update(data.encode('utf-8'))
    return sha256.hexdigest()

# Example usage
data = "Hello, World!"
hashed_data = hash_data(data)
print(f"Data: {data}")
print(f"SHA256 Hash: {hashed_data}")


Data: Hello, World!
SHA256 Hash: dffd6021bb2bd5b0af676290809ec3a53191dd81c7f70a4b28688a362182986f


Diffie-Hellman Key Exchange

In [5]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives import serialization

def generate_dh_parameters():
    # Generate Diffie-Hellman parameters
    parameters = dh.generate_parameters(generator=2, key_size=2048, backend=default_backend())
    return parameters

def generate_dh_private_key(parameters):
    # Generate a private key for these parameters
    private_key = parameters.generate_private_key()
    return private_key

def generate_dh_public_key(private_key):
    # Generate a public key corresponding to the private key
    public_key = private_key.public_key()
    return public_key

def get_shared_secret(private_key, peer_public_key):
    # Derive the shared secret
    shared_secret = private_key.exchange(peer_public_key)
    return shared_secret

# Example usage
parameters = generate_dh_parameters()
private_key1 = generate_dh_private_key(parameters)
public_key1 = generate_dh_public_key(private_key1)

private_key2 = generate_dh_private_key(parameters)
public_key2 = generate_dh_public_key(private_key2)

# Exchange and derive the shared secret
shared_secret1 = get_shared_secret(private_key1, public_key2)
shared_secret2 = get_shared_secret(private_key2, public_key1)

assert shared_secret1 == shared_secret2  # Both parties should have the same shared secret


RSA Implementation

In [6]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes

In [7]:
def generate_rsa_keys():
    # Generate private key
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    # Generate public key
    public_key = private_key.public_key()
    return private_key, public_key

def rsa_encrypt(public_key, plaintext):
    # Encrypt plaintext with the public key
    ciphertext = public_key.encrypt(
        plaintext,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return ciphertext

def rsa_decrypt(private_key, ciphertext):
    # Decrypt ciphertext with the private key
    plaintext = private_key.decrypt(
        ciphertext,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    return plaintext

# Example usage
private_key, public_key = generate_rsa_keys()

message = b"Hello, RSA!"
ciphertext = rsa_encrypt(public_key, message)
print("Encrypted:", ciphertext)

decrypted_message = rsa_decrypt(private_key, ciphertext)
print("Decrypted:", decrypted_message)


Encrypted: b'd\xbf~\x04_\x7f9\x8d=\x96\x8f\x13;\xd4iy\t\xfa?\x08&\xce,\x8aI\x92h>\x860\t~\xe5\xbbf\xc4\xc4\xfauV\xa0\x8a`\xcf\x19]\xf9?\xe7`\x99\xc1\x7f`<Z\x001\x9e\x04\x98\\\x0c\x82,Ztp\x1a\x02\xfcS\xcc\nM\xd3\xf4;\xe3\x9az\xce\xf0\xacr\xc5Oa\xaaS\xce`\xddh\xcc\xfaO\x00F\xbb\x10cn\xa4\xfc!\xe6=\x13\xb5uGdA\x05\x11N\xbaB\x05Z\x12\xb9\xef\x82\xfdV\xb7\xde\x96\x1fH\x15\xacBw\xa1\\\xf46\xcf\xb4g\x8c\xdb\x0b\x8a"\xa8\xb13\xd7\x03\x8e\x05\xcc\xb1 #\xb8\x04\x11\xe7\xfc\x8d\xac\x18\x0eS\x15\xaeU\xcbe\x06@\xb1\xbc\xe5[\xdf[P\x86\xfe\x85K\x16\xf6%\xd5>X\xfeT\xa0\x0f\xd9oa\x04\xf7\xe5\xe2\x01\x9a\x0boI\xc7\x02\n\x05\xe1\x04\x89w\t=(\xa5.\x8cP\x0f\xeb8?RtH\x11\xba\x8e\xc3!\xd4\xcf\x9a\xad\x1b{P;\x86\x1bdWp\x10`F-\xa8r='
Decrypted: b'Hello, RSA!'


Basic PKI System

1. Certificate Generation and Signing
We'll create a self-signed certificate for simplicity. In a real-world scenario, you would have a Certificate Authority (CA) to sign the certificates.

In [8]:
from cryptography import x509
from cryptography.x509.oid import NameOID
from datetime import datetime, timedelta
import ipaddress

def generate_self_signed_cert(private_key, public_key):
    subject = issuer = x509.Name([
        x509.NameAttribute(NameOID.COUNTRY_NAME, u"US"),
        x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, u"California"),
        x509.NameAttribute(NameOID.LOCALITY_NAME, u"San Francisco"),
        x509.NameAttribute(NameOID.ORGANIZATION_NAME, u"My Company"),
        x509.NameAttribute(NameOID.COMMON_NAME, u"mysite.com"),
    ])
    cert = x509.CertificateBuilder().subject_name(
        subject
    ).issuer_name(
        issuer
    ).public_key(
        public_key
    ).serial_number(
        x509.random_serial_number()
    ).not_valid_before(
        datetime.utcnow()
    ).not_valid_after(
        # Our certificate will be valid for 10 days
        datetime.utcnow() + timedelta(days=10)
    ).add_extension(
        x509.SubjectAlternativeName([x509.DNSName(u"localhost")]),
        critical=False,
    # Sign our certificate with our private key
    ).sign(private_key, hashes.SHA256(), default_backend())
    return cert

# Example usage
private_key, public_key = generate_rsa_keys()
cert = generate_self_signed_cert(private_key, public_key)
print(cert)


<Certificate(subject=<Name(C=US,ST=California,L=San Francisco,O=My Company,CN=mysite.com)>, ...)>


2. Certificate Verification
In a real-world scenario, you would verify the certificate against the CA's public key. Since we are using a self-signed certificate, we can verify it using its own public key.

In [9]:
def verify_certificate(cert, public_key):
    # In a real-world scenario, you would use the CA's public key
    try:
        public_key.verify(
            cert.signature,
            cert.tbs_certificate_bytes,
            padding.PKCS1v15(),
            cert.signature_hash_algorithm,
        )
        return True
    except Exception as e:
        print("Verification failed:", e)
        return False

# Example usage
is_verified = verify_certificate(cert, public_key)
print("Certificate verified:", is_verified)


Certificate verified: True


Client-Server Architecture

Server Setup
The server will listen for incoming connections, perform the SSL/TLS handshake, and then communicate securely with the client.

In [10]:
import socket
import threading

def handle_client(client_socket, client_address):
    print(f"Accepted connection from {client_address}")

    # Perform SSL/TLS handshake here (to be implemented)

    # Secure communication (to be implemented)

    client_socket.close()

def start_server(host, port):
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.bind((host, port))
    server.listen(5)
    print(f"Listening on {host}:{port}")

    try:
        while True:
            client_socket, client_address = server.accept()
            client_thread = threading.Thread(target=handle_client, args=(client_socket, client_address))
            client_thread.start()
    except KeyboardInterrupt:
        print("Server shutting down.")
        server.close()

# Example usage
HOST = '127.0.0.1'
PORT = 65432
start_server(HOST, PORT)


Listening on 127.0.0.1:65432
Server shutting down.


Simplified Handshake Protocol
The handshake will involve the following steps:

Client Hello: The client sends a hello message with its RSA public key.

Server Hello: The server responds with its RSA public key.

Key Exchange: Both parties use the received public key to encrypt a generated
AES key and send it to the other party.

Establishment of a Secure Channel: Both parties decrypt the received AES key and use it for further communication.

Server-Side Handshake
Here's how the server-side handshake can be implemented:

In [13]:
import socket
import threading


def handle_client(client_socket, client_address):
    print(f"Accepted connection from {client_address}")

    # Perform SSL/TLS handshake
    aes_key = server_handshake(client_socket)
    aes_cipher = AESCipher(aes_key)

    # Secure communication
    encrypted_message = receive_data(client_socket)
    message = aes_cipher.decrypt(encrypted_message)
    print(f"Received from {client_address}: {message}")

    client_socket.close()

def start_server(host, port):
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.bind((host, port))
    server.listen(5)
    print(f"Listening on {host}:{port}")

    try:
        while True:
            client_socket, client_address = server.accept()
            client_thread = threading.Thread(target=handle_client, args=(client_socket, client_address))
            client_thread.start()
    except KeyboardInterrupt:
        print("Server shutting down.")
        server.close()

# Example usage
HOST = '127.0.0.1'
PORT = 65432
start_server(HOST, PORT)


Listening on 127.0.0.1:65432
Server shutting down.


In [14]:
def server_handshake(client_socket):
    # Receive client's public key
    client_public_key = receive_public_key(client_socket)

    # Send server's public key
    server_private_key, server_public_key = generate_rsa_keys()
    send_public_key(client_socket, server_public_key)

    # Receive encrypted AES key from client and decrypt it
    encrypted_aes_key = receive_data(client_socket)
    aes_key = rsa_decrypt(server_private_key, encrypted_aes_key)

    return aes_key

def send_public_key(socket, public_key):
    # Serialize and send the public key
    pem = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )
    socket.sendall(pem)

def receive_public_key(socket):
    # Receive and deserialize the public key
    pem = socket.recv(2048)
    public_key = serialization.load_pem_public_key(pem, backend=default_backend())
    return public_key

def receive_data(socket):
    # Receive data
    data = socket.recv(2048)
    return data

Client-Side Handshake
And here's the client-side handshake:

In [ ]:
def client_handshake(server_socket):
    # Generate client's RSA keys and send public key
    client_private_key, client_public_key = generate_rsa_keys()
    send_public_key(server_socket, client_public_key)

    # Receive server's public key
    server_public_key = receive_public_key(server_socket)

    # Generate AES key, encrypt with server's public key, and send
    aes_key = os.urandom(32)  # AES-256 key
    encrypted_aes_key = rsa_encrypt(server_public_key, aes_key)
    send_data(server_socket, encrypted_aes_key)

    return aes_key

def start_client(host, port):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as client:
        client.connect((host, port))

        # Perform SSL/TLS handshake
        aes_key = client_handshake(client)
        aes_cipher = AESCipher(aes_key)

        # Secure communication
        message = "Hello, secure world!"
        encrypted_message = aes_cipher.encrypt(message.encode())
        send_data(client, encrypted_message)

# Example usage
HOST = '127.0.0.1'
PORT = 65432
start_client(HOST, PORT)
